# Setup

In [ ]:
import matplotlib
matplotlib.use('Agg')

In [ ]:
%matplotlib notebook

In [ ]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', 30) # 27 columns of data in training set

# import tensorflow as tf
import matplotlib.pyplot as plt

# from sklearn import datasets, cross_validation, metrics
from sklearn.cross_validation import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import Imputer
# from sklearn import preprocessing

# Random Forest
from sklearn.ensemble import RandomForestClassifier

# from sklearn.grid_search import GridSearchCV

In [ ]:
def make_features(data):    
    '''
    Extract date-time features from dataframe 'data'.
    Converts date_time, srch_ci, and srch_co fields into
    components (year, month, day, etc) and drops the 
    original field.
    '''
    extract_datetimes(data, 'date_time')
    extract_datetimes(data, 'srch_ci')
    extract_datetimes(data, 'srch_co')
    
    data = data.drop(['date_time', 'srch_ci', 'srch_co'], axis=1)
    
    has_null = ['orig_destination_distance', 'srch_ci_year', 'srch_ci_month', 
                'srch_ci_day', 'srch_ci_hour', 'srch_ci_minute', 
                'srch_ci_dayofyear', 'srch_ci_dayofweek', 'srch_co_year', 
                'srch_co_month', 'srch_co_day', 'srch_co_hour', 'srch_co_minute',
                'srch_co_dayofyear', 'srch_co_dayofweek']
    imp = Imputer(missing_values='NaN', strategy='mean', axis=0, copy=False)
    data[has_null] = imp.fit_transform(data[has_null])
    
    return data

def extract_datetimes(data, field):
    data[field] = pd.to_datetime(data[field],errors='coerce')
    data[field+'_year'] = data[field].dt.year
    data[field+'_month'] = data[field].dt.month
    data[field+'_day'] = data[field].dt.day
    data[field+'_hour'] = data[field].dt.hour
    data[field+'_minute'] = data[field].dt.minute
    data[field+'_dayofyear'] = data[field].dt.dayofyear
    data[field+'_dayofweek'] = data[field].dt.dayofweek

def make_PCA(X, n_comp):
    pca = PCA(n_components=n_comp)
    pca.fit(X)
    return pca

In [ ]:
use_PCA = True
save_preds = True

# Make features

In [ ]:
sample_sub = pd.read_csv('data/sample_submission.csv')
train_data = make_features(pd.read_csv('data/train.csv', nrows=50000)) # 37,670,294 total lines
test_data = make_features(pd.read_csv('data/test.csv'))   # 2,528,244 total lines
destinations_data = pd.read_csv('data/destinations.csv')

In [ ]:
# train_data['srch_destination_id'][0]

In [ ]:
# destinations_data[destinations_data['srch_destination_id']==train_data['srch_destination_id'][3]]

In [ ]:
features = ['site_name', 'posa_continent', 'user_location_country',
       'user_location_region', 'user_location_city',
       'orig_destination_distance', 'user_id', 'is_mobile', 'is_package',
       'channel', 'srch_adults_cnt', 'srch_children_cnt', 'srch_rm_cnt',
       'srch_destination_id', 'srch_destination_type_id',
       'hotel_continent', 'hotel_country', 'hotel_market',
       'date_time_year', 'date_time_month', 'date_time_day', 'date_time_hour',
       'date_time_minute', 'date_time_dayofyear', 'date_time_dayofweek',
       'srch_ci_year', 'srch_ci_month', 'srch_ci_day', 'srch_ci_hour',
       'srch_ci_minute', 'srch_ci_dayofyear', 'srch_ci_dayofweek',
       'srch_co_year', 'srch_co_month', 'srch_co_day', 'srch_co_hour',
       'srch_co_minute', 'srch_co_dayofyear', 'srch_co_dayofweek']

# 'srch_ci', 'srch_co', 'orig_destination_distance', 'is_booking', 'cnt',

In [ ]:
X_all = train_data.ix[:,features]
y_all = train_data.ix[:,'hotel_cluster']
X_test = test_data.ix[:,features]

In [ ]:
# # checking correlation of features
# plt.matshow(X_all.corr())
# plt.colorbar()

In [ ]:
if use_PCA:
    X = X_all.as_matrix()
    pca = make_PCA(X, 15)
    X = pca.transform(X)

In [ ]:
# generate training and cross-validation features
X_train, X_cv, y_train, y_cv = train_test_split(X, y_all, train_size=.99, random_state=1)

# Train: Random Forest Model

In [ ]:
expedia_rfc = RandomForestClassifier(n_estimators=100, 
                                     max_leaf_nodes=6, 
                                     criterion='entropy')

In [ ]:
%time expedia_rfc = expedia_rfc.fit(X_train, y_train)

In [ ]:
%%time

score_train = expedia_rfc.score(X_train, y_train)
score_cv = expedia_rfc.score(X_cv, y_cv)

# train/cv
print ('Training Score:', score_train, ', CV Score:', score_cv) 

In [ ]:
feature_importance = zip(features, expedia_rfc.feature_importances_)
for x in sorted(feature_importance, key=lambda x: -x[1]):
    print (x)

# Predict

In [ ]:
if use_PCA:
    X_test = pca.transform(X_test.as_matrix())

In [1]:
# break the test set into n = num_split sets to predict on
split_size = int(X_test.shape[0]*.1)
num_splits = 10
n_test = X_test.shape[0]
top_pred_hotel_cluster = []

NameError: name 'X_test' is not defined

In [ ]:
%%time

for i in range(num_splits):
    prob_prediction = expedia_rfc.predict_proba(X_test[int(i*n_test/num_splits):int((i+1)*n_test/num_splits)])
    top_pred_hotel_cluster.append([' '.join([str(hotel) for hotel in row]) for row in np.argsort(prob_prediction)[:,-5:]])

In [ ]:
%%time 
prob_prediction = expedia_rfc.predict_proba(X_test[:split_size])
top_pred_hotel_cluster.append([' '.join([str(hotel) for hotel in row]) for row in np.argsort(prob_prediction)[:,-5:]])

In [ ]:
a = np.array(top_pred_hotel_cluster)

In [ ]:
submission = pd.DataFrame(np.concatenate((a[0],a[1],a[2],a[3],a[4],a[5],a[6],a[7],a[8],a[9]), axis=0), columns=['hotel_cluster'])

In [ ]:
submission.to_csv('expedia-rf-2016-04-23-s1.csv', index_label='Id')